In [ ]:
source(paste0(dirname(dirname(dirname(getwd()))),'/map.r'))
source(paste0(HELP_DIR, "shortcuts.r"))
source(paste0(HELP_DIR, "helpers.r"))

# 0 - Ready prepared cohort counts

In [ ]:
top_mechanisms <- fread(paste0(SHARE_DIR, "top_mechanisms.csv"))

In [ ]:
go <- 
top_mechanisms %>% 
 mu(pan = factor(ifelse(m("Pan-Cancer", cohortGo), "Pan-Cancer", "Cohort Specific"), levels = c("Pan-Cancer", "Cohort Specific"))) %>% 
 rw() %>% 
 mu(cohortShow = ifelse(pan == "Pan-Cancer", paste0(strsplit(cohortGo, " ## ")[[1]][-1], collapse = "/"), gsub(" ## ", "/", cohortGo)),
    cohortShow = gsub(" \\(ant\\)agonist", "", cohortShow), cohortShow = gsub("Topoisomerase inhibitor", "TI", cohortShow)) %>% 
 mu(treatment = paste0(strsplit(cohortGo, " ## ")[[1]][-1], collapse = "/"), 
    cohort = paste0(strsplit(cohortGo, " ## ")[[1]][1]), 
    cohort = ifelse(cohort == "Pan-Cancer", "", cohort)) %>% 
 ug() 

# 1 - Figure

In [ ]:
colors <- c("No Durable Clinical Benefit" = "#e52f28", "Durable Clinical Benefit" = "#7AABD3")

In [ ]:
options(repr.plot.width = 12, repr.plot.height = 8)

In [ ]:
go %>% 
 ga(response, patients, -cohortShow, -ct, -pan, -cohortGo, -cohort, -treatment) %>% 
 mu(response = ifelse(response == "dcb", "Durable Clinical Benefit", "No Durable Clinical Benefit")) %>% 
 ggplot(aes(y = fct_reorder(cohortShow, patients), x = patients, fill = response)) + 
 geom_bar(stat = "identity", color = "black") + 
 scale_fill_manual( values = colors) +  
 go_theme + 
 labs(x = "# of Patients", y = "Post-Biopsy Treatment Mechanism", title = "Available Cohort Sizes", fill = NULL) + 
 facet_grid(pan ~., scales = "free_y", space = "free") + 
 geom_text(aes(label = patients), position = position_stack(vjust = 0.5), color = "black", size = 6) +
 theme(strip.text = element_text(size = 21, color = "black"), axis.text.y = element_text(size = 21), axis.text.x = element_text(size = 21)) + 
 theme_bw(base_size = 22) + theme( panel.grid.major = element_blank(), panel.grid.minor = element_blank()) + 
 theme(legend.position = c(0.97, .13),  legend.justification = c(1, 0))

In [ ]:
ggsave( paste0(FIG_DIR, "cohort_counts.png"), width = 12, height = 8, dpi = 500)